In [2]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, matthews_corrcoef, auc, precision_recall_curve, make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import igraph as ig
import itertools
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [6]:
pos_y = "Data/Labels/Tclin.csv"
pos_y2 = "Data/Labels/pancreatic intraductal papillary-mucinous neoplasm-Tclin.csv"
pos_y3 = "Data/Labels/acute myeloid leukemia-tclin.csv"


graph_ = "Data/Network/raw_Tclin_Signalink_PIN_graph.graphml"
graph = ig.Graph.Load(graph_, format='graphml')

data = []
for vertex in graph.vs:
    # Get node attributes excluding 'label'
    node_data = {key: vertex[key] for key in vertex.attributes() if key not in ['label', 'id']}
    
    # Ensure 'name' is the first key
    if 'name' in node_data:
        node_data = {'Gene_name': node_data.pop('name'), **node_data}
    
    data.append(node_data)

# Convert to DataFrame
x = pd.DataFrame(data)


imgagn_ = "Data/Network/ImGAGN_graph_tclin.graphml"
imgagn = ig.Graph.Load(imgagn_, format='graphml')
a = imgagn.vs['name']
a = pd.DataFrame(a)

imgagn_embed = pd.read_csv('Model/ImGAGN/Tclin-Imgagn-Embedding.csv')

imgagn_embed_ = pd.concat((a, imgagn_embed.iloc[:6048, :]), axis=1)

imgagn_embed_.columns = ['Gene_name'] + list(imgagn_embed_.columns[1:])
x.columns = ['Gene_name'] + list(x.columns[1:])

# Merge the dataframes on the 'key' column, keeping only matching rows
X = pd.merge(x, imgagn_embed_, on='Gene_name', how='inner')

X

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Network/ImGAGN_graph_tclin.graphml'

In [111]:
def split_balance(df, n_splits=20, random_state=42, balance_ratio=1):
    np.random.seed(random_state)
    positive_samples = df[df['label'] == 1]
    negative_samples = df[df['label'] == 0]
    
    
    folds = []
    
    for _ in range(n_splits):
        seed = random_state + _
        # Select 80% of positive samples
        pos_fold = positive_samples.sample(frac=0.8, random_state=seed)
        
        # Calculate the number of negative samples needed
        num_neg_samples = int(len(pos_fold) * balance_ratio)
        
        # Randomly select negative samples
        neg_fold = negative_samples.sample(n=num_neg_samples, random_state=seed)
        
        # Combine positive and negative samples
        fold = pd.concat([pos_fold, neg_fold])
        
        # Shuffle the fold
        fold = fold.sample(frac=1, random_state=seed).reset_index(drop=True)
        
        folds.append(fold)
    
    return folds

def aupr_score(y_true, y_pred_proba):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    return auc(recall, precision)

def compute_metrics(y_true, y_proba, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_proba),
        'aupr': aupr_score(y_true, y_proba)
    }


def custom_cross_validate(X, hype=None, n_splits=5, spliting=20, random_state=42, balance_ratio=1, classifier='XgBoost'):
    # Initialize KFold cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    all_metrics = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        
        x_df_train = pd.DataFrame(X_train)
        x_df_test = pd.DataFrame(X_test)

        folds = split_balance(x_df_train, n_splits=spliting, balance_ratio=balance_ratio)
        models = train(folds, hype=hype, classifier=classifier)
        pre_proba = evaluate(x_df_test, models)
        
        y_pred = (pre_proba >= 0.5).astype(int)
        
        metrics = compute_metrics(x_df_test['label'], pre_proba, y_pred)
        all_metrics.append(metrics)

    return all_metrics

def train(folds, hype=None, classifier='XgBoost'):

    models = []

    for i in range(len(folds)):

        X_features = folds[i].drop(['Gene_name'], axis=1)
        y = X_features['label']
        X_features = X_features.drop('label', axis=1)

        if classifier=='XgBoost':
            # Define the XGBoost model
            if hype != None:
                model = xgb.XGBClassifier(**hype)
                model.fit(X_features, y)
            else:
                model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
                model.fit(X_features, y)
                
        elif classifier == 'DT':
            
            if hype is not None:
                model = DecisionTreeClassifier(**hype)
            else:
                model = DecisionTreeClassifier(random_state=42)
            model.fit(X_features, y)

        elif classifier == 'RF':
            
            # Define the Random Forest model
            if hype is not None:
                model = RandomForestClassifier(**hype)
            else:
                model = RandomForestClassifier(random_state=42)
            model.fit(X_features, y)

        models.append(model)

    return models

def evaluate(df_test, models):

    test_features = df_test.drop(['Gene_name'], axis=1)
    y = test_features['label']
    test_features = test_features.drop('label', axis=1)
    predictions_proba = []

    
    for model in models:
        # Predict probabilities
        proba = model.predict_proba(test_features)[:, 1]  # Probabilities for the positive class
        predictions_proba.append(proba)

    predictions_proba = np.array(predictions_proba)
    avg_probas = np.mean(predictions_proba, axis=0)

    return avg_probas

def compute_mean_and_std_metrics(metrics):
    # Initialize dictionaries to hold the sums and squared sums of each metric
    metric_sums = {key: 0 for key in metrics[0].keys()}
    metric_squared_sums = {key: 0 for key in metrics[0].keys()}

    # Sum each metric and squared metric across all folds
    for metric in metrics:
        for key, value in metric.items():
            metric_sums[key] += value
            metric_squared_sums[key] += value ** 2

    n = len(metrics)
    # Calculate the mean for each metric
    metric_means = {key: value / n for key, value in metric_sums.items()}

    # Calculate the standard deviation for each metric
    metric_stds = {
        key: ((metric_squared_sums[key] / n) - (metric_means[key] ** 2)) ** 0.5 for key in metrics[0].keys()
    }

    # Construct the string with mean ± std for each metric
    metric_results = {key: f"{metric_means[key]:.4f} ± {metric_stds[key]:.4f}" for key in metrics[0].keys()}

    return metric_results

def pipeline(X, pos_y, hype=None, cross_split=5, balance_split=30, balance_ratio=1, classifier='XgBoost'):

    positive_labels = pd.read_csv(pos_y, header=None)
    X['label'] = X['Gene_name'].isin(positive_labels[1]).astype(int)
    result = custom_cross_validate(X, hype=hype, n_splits=cross_split, spliting=balance_split, balance_ratio=balance_ratio, classifier=classifier)
    met = compute_mean_and_std_metrics(result)

    for keys, value in met.items():
        print(keys + ': ' + value)

    return met

def pred(X, pos_y, hype=None, balance_split=30, balance_ratio=1, classifier='XgBoost'):

    positive_labels = pd.read_csv(pos_y, header=None)
    X['label'] = X['Gene_name'].isin(positive_labels[1]).astype(int)

    X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
        
    x_df_train = pd.DataFrame(X_train)
    x_df_test = pd.DataFrame(X_test)
    
    folds = split_balance(x_df_train, n_splits=balance_split, balance_ratio=balance_ratio)
    models = train(folds, hype=hype, classifier=classifier)

    test_features = X.drop(['Gene_name'], axis=1)
    test_features = test_features.drop('label', axis=1)
    predictions_proba = []

    
    for model in models:
        # Predict probabilities
        proba = model.predict_proba(test_features)[:, 1]  # Probabilities for the positive class
        predictions_proba.append(proba)

    predictions_proba = np.array(predictions_proba)
    avg_probas = np.mean(predictions_proba, axis=0)

    return avg_probas


def grid_search(X, pos_y, param_grid, cross_split=5, balance_split=30, balance_ratio=1, classifier='XgBoost'):
    # Generate all combinations of hyperparameters
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    results = []
    trial = 0
    for params in combinations:
        trial +=1
        # Update the hype dictionary with the current combination of hyperparameters

        #XGB
        '''
        hype = {
            'learning_rate': params['learning_rate'],  # Different learning rates
            'max_depth': params['max_depth'],               # Different depths of trees
            'n_estimators': params['n_estimators'],           # Number of trees
            'subsample': params['subsample'],             # Fraction of samples used for fitting
            'colsample_bytree': params['colsample_bytree'] # Whether bootstrap samples are used when building trees
        }
        '''

        #RF
        #
        hype = {
            'n_estimators': params['n_estimators'],  # Different learning rates
            'max_depth': params['max_depth'],               # Different depths of trees
            'min_samples_split': params['min_samples_split'],           # Number of trees
            'max_features': params['max_features'],             # Fraction of samples used for fitting
            'bootstrap': params['bootstrap'] # Whether bootstrap samples are used when building trees
        }
        

        #DT
        '''
        hype = {
            'criterion': params['criterion'],  # Different learning rates
            'max_depth': params['max_depth'],               # Different depths of trees
            'min_samples_split': params['min_samples_split'],           # Number of trees
            'min_samples_leaf': params['min_samples_leaf'],             # Fraction of samples used for fitting
            'splitter': params['splitter'] # Whether bootstrap samples are used when building trees
        }
        '''

        # Run the pipeline with the current hyperparameters
        met = pipeline(X, pos_y, hype, cross_split=cross_split, balance_split=balance_split, balance_ratio=balance_ratio, classifier=classifier)
        score = float(met['roc_auc'][:-9])

        # Store the result
        print(f"trial: {trial}: score: {score}")
        results.append({'params': params, 'score': score})

    # Sort results by score in descending order
    results = sorted(results, key=lambda x: x['score'], reverse=True)

    best_params = results[0]['params']
    best_score = results[0]['score']

    return best_params, best_score, results

In [112]:
param_grid_xgb = {
    'learning_rate': [0.01, 0.05, 0.1],  # Different learning rates
    'max_depth': [10],               # Different depths of trees
    'n_estimators': [300],           # Number of trees
    'subsample': [0.6, 1.0],             # Fraction of samples used for fitting
    'colsample_bytree': [0.6, 1.0]       # Fraction of features used for fitting
}

param_grid_dt = {
    'criterion': ['entropy'],            # Criterion for splitting ('gini' or 'entropy')
    'max_depth': [3, 7, 10, 12],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 20, 30],         # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 12],             # Minimum number of samples required to be at a leaf node
    'splitter': ['best']
}

param_grid_rf = {
    'n_estimators': [300],            # Number of trees in the forest
    'max_depth': [15],            # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],           # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],             # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt'],    # Number of features to consider when looking for the best split
    'bootstrap': [False]                 # Whether bootstrap samples are used when building trees
}

param, score, log = grid_search(X, pos_y, param_grid_rf, cross_split=5, balance_split=30, balance_ratio=2, classifier='RF')

param, score

accuracy: 0.8967 ± 0.0086
precision: 0.0962 ± 0.0236
recall: 0.6786 ± 0.0774
f1: 0.1670 ± 0.0356
roc_auc: 0.8900 ± 0.0304
aupr: 0.1627 ± 0.0343
trial: 1: score: 0.89
accuracy: 0.8967 ± 0.0086
precision: 0.0962 ± 0.0236
recall: 0.6786 ± 0.0774
f1: 0.1670 ± 0.0356
roc_auc: 0.8900 ± 0.0304
aupr: 0.1627 ± 0.0343
trial: 2: score: 0.89
accuracy: 0.8967 ± 0.0086
precision: 0.0962 ± 0.0236
recall: 0.6786 ± 0.0774
f1: 0.1670 ± 0.0356
roc_auc: 0.8900 ± 0.0304
aupr: 0.1627 ± 0.0343
trial: 3: score: 0.89
accuracy: 0.8957 ± 0.0075
precision: 0.0942 ± 0.0231
recall: 0.6710 ± 0.0943
f1: 0.1636 ± 0.0350
roc_auc: 0.8872 ± 0.0297
aupr: 0.1624 ± 0.0342
trial: 4: score: 0.8872
accuracy: 0.8957 ± 0.0075
precision: 0.0942 ± 0.0231
recall: 0.6710 ± 0.0943
f1: 0.1636 ± 0.0350
roc_auc: 0.8872 ± 0.0297
aupr: 0.1624 ± 0.0342
trial: 5: score: 0.8872
accuracy: 0.8957 ± 0.0075
precision: 0.0942 ± 0.0231
recall: 0.6710 ± 0.0943
f1: 0.1636 ± 0.0350
roc_auc: 0.8872 ± 0.0297
aupr: 0.1624 ± 0.0342
trial: 6: score: 0.887

({'n_estimators': 300,
  'max_depth': 15,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'max_features': 'sqrt',
  'bootstrap': False},
 0.89)

In [113]:
params_xgb = {'learning_rate': 0.01,
  'max_depth': 10,
  'n_estimators': 300,
  'subsample': 0.6,
  'colsample_bytree': 1.0}

params_dt = {
    'criterion': 'entropy',            # Criterion for splitting ('gini' or 'entropy')
    'max_depth': 7,     # Maximum depth of the tree
    'min_samples_split': 20,         # Minimum number of samples required to split an internal node
    'min_samples_leaf': 1,             # Minimum number of samples required to be at a leaf node
    'splitter': 'best'
}

params_rf = {'n_estimators': 300,
  'max_depth': 15,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'max_features': 'sqrt',
  'bootstrap': False}

met = pipeline(X, pos_y, hype=params_rf, cross_split=5, balance_split=30, balance_ratio=2, classifier='RF')

met


accuracy: 0.9038 ± 0.0106
precision: 0.1023 ± 0.0246
recall: 0.6726 ± 0.0822
f1: 0.1761 ± 0.0367
roc_auc: 0.8975 ± 0.0306
aupr: 0.1883 ± 0.0656


{'accuracy': '0.9038 ± 0.0106',
 'precision': '0.1023 ± 0.0246',
 'recall': '0.6726 ± 0.0822',
 'f1': '0.1761 ± 0.0367',
 'roc_auc': '0.8975 ± 0.0306',
 'aupr': '0.1883 ± 0.0656'}

In [ ]:
########Further Evaluations:######

In [69]:
avg_prob = pred(X, pos_y, hype=params_xgb, balance_split=30, balance_ratio=2, classifier='XgBoost')

KeyboardInterrupt: 

In [ ]:
names = X['Gene_name']
probs = pd.DataFrame(avg_prob)

final_prediction = pd.concat((names, probs), axis=1)

final_prediction


,Gene_name,0
0,A1BG,0.300617
1,A2M,0.872885
2,AAAS,0.006333
3,AAMP,0.108457
4,AANAT,0.170956
...,...,...
6043,ZSCAN22,0.001012
6044,ZW10,0.000604
6045,ZXDC,0.000573
6046,ZYG11B,0.000562


In [ ]:
sorted_df = final_prediction.sort_values(by=final_prediction.columns[1], ascending=False)  # Use ascending=True for ascending order

# Save the sorted dataframe to a new CSV file
sorted_csv_file_path = "final_prediction.csv"  # Replace with your desired file path
sorted_df.to_csv(sorted_csv_file_path, index=False)

In [ ]:
sorted_df

,Gene_name,0
3151,MET,0.997300
1894,FLT3,0.996389
194,ALK,0.996287
978,CHRM3,0.996285
2423,IGF1R,0.996206
...,...,...
2710,KCTD10,0.000270
6027,ZNF721,0.000258
2768,KLHDC2,0.000250
1932,FRMD8,0.000235


In [ ]:
# Calculate the number of rows corresponding to the top 5%
top_5_percent_count = int(len(sorted_df) * 0.05)

# Select the top 5% rows
top_5_percent_df = sorted_df.head(top_5_percent_count)

# Convert the top 5% rows to a list of dictionaries
top_5_percent_list = top_5_percent_df.to_dict(orient='records')

# Print the list
print(top_5_percent_list)

# Optionally, save the top 5% dataframe to a new CSV file
top_5_percent_csv_file_path = "top_5_percent_final_prediction.csv"  # Replace with your desired file path
top_5_percent_df.to_csv(top_5_percent_csv_file_path, index=False)

print("Top 5% predictions CSV saved successfully.")

[{'Gene_name': 'MET', 0: 0.9973000884056091}, {'Gene_name': 'FLT3', 0: 0.9963890910148621}, {'Gene_name': 'ALK', 0: 0.9962866306304932}, {'Gene_name': 'CHRM3', 0: 0.9962854981422424}, {'Gene_name': 'IGF1R', 0: 0.996205747127533}, {'Gene_name': 'JAK1', 0: 0.9959810376167297}, {'Gene_name': 'JAK2', 0: 0.9957574605941772}, {'Gene_name': 'ADORA2A', 0: 0.9957215785980225}, {'Gene_name': 'PDGFRB', 0: 0.9953297972679138}, {'Gene_name': 'KIT', 0: 0.9953213930130005}, {'Gene_name': 'HTR2B', 0: 0.9951905012130737}, {'Gene_name': 'FGFR2', 0: 0.9951538443565369}, {'Gene_name': 'SYK', 0: 0.9948833584785461}, {'Gene_name': 'PDGFRA', 0: 0.9948533177375793}, {'Gene_name': 'NDUFS3', 0: 0.994557797908783}, {'Gene_name': 'HDAC3', 0: 0.9943109750747681}, {'Gene_name': 'ABL1', 0: 0.9942794442176819}, {'Gene_name': 'SLC6A3', 0: 0.9941022396087646}, {'Gene_name': 'FGFR1', 0: 0.9940539002418518}, {'Gene_name': 'ERBB3', 0: 0.994002640247345}, {'Gene_name': 'ACE', 0: 0.9939194917678833}, {'Gene_name': 'ITGB3', 